In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from cytocherrypick.calculations import find_median
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(f"../results/{CELL_TYPE}/single_cell_predictions.parquet").resolve()

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_Nuclei_Location_Center_Y,Metadata_Nuclei_Location_Center_X,group,Nuclei_RadialDistribution_ZernikePhase_CorrGasdermin_3_1,Cytoplasm_RadialDistribution_ZernikePhase_CorrGasdermin_9_1,Cytoplasm_AreaShape_Zernike_9_5,Metadata_Well,Metadata_Site,Metadata_ImageNumber,Metadata_Cells_Number_Object_Number,comparison,treatment,feature,image_compsite_path,image_composite_crop_path,image_DAPI_path,image_ER_path,image_GasderminD_path,image_AGP_path,image_Mitochondria_path,image_compsite_crop_path,image_DAPI_crop_path,image_ER_crop_path,image_GasderminD_crop_path,image_AGP_crop_path,image_Mitochondria_crop_path
0,media_ctr_0.0_0_Media_ctr_0.0_0,149.170673,550.463942,Control,1.732595,0.246475,-0.640791,O04,7,215,189,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrGas...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
1,DMSO_0.100_%_DMSO_1.000_%,643.401361,1612.034014,Control,1.732594,-0.387799,-0.384997,O12,1,337,1066,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrGas...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
2,DMSO_0.100_%_DMSO_0.025_%,791.517110,1769.718631,Control,-1.728316,0.050109,1.992991,I07,11,267,1621,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrGas...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
3,H2O2_100.000_uM_DMSO_0.025_%,789.768421,180.649123,Control,-1.728317,-0.349258,0.574726,I04,6,214,1379,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrGas...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
4,media_ctr_0.0

In [4]:
df.columns
image_path_columns = [
    "image_compsite_path",
    "image_DAPI_path",
    "image_ER_path",
    "image_GasderminD_path",
    "image_AGP_path",
    "image_Mitochondria_path",
    "image_composite_crop_path",
    "image_DAPI_crop_path",
    "image_ER_crop_path",
    "image_GasderminD_crop_path",
    "image_AGP_crop_path",
    "image_Mitochondria_crop_path",
]
for path_type in image_path_columns:
    print(path_type)
    for image_path in df[path_type]:
        print(image_path)
        # open the image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        # image to array
        image_array = np.array(image)
        resolution = 3.3446  # pixels per micrometer
        # get the image size
        image_size = image_array.shape
        if image_array.shape[1] <= 100:
            scale_bar_length = 5  # um
            scale_bar_height = 1  # pixels
            padding = 3  # pixels
        elif image_array.shape[1] > 100:
            scale_bar_length = 100
            scale_bar_height = 10
            padding = 10
        # get the bottom right most corner based on scale 1 % of pixels
        scale_bar_x = (
            image_size[1] - (scale_bar_length * resolution) - padding - padding
        )
        scale_bar_y = image_size[0] - (scale_bar_height) - padding
        print(scale_bar_x, scale_bar_y)
        # draw the scale bar
        new = cv2.rectangle(
            image_array,
            (int(scale_bar_x), int(scale_bar_y)),
            (image_size[1] - padding, image_size[0] - padding),
            (255, 255, 255),
            -1,
        )
        # show the image
        # plt.imshow(new)
        # plt.show()
        # save the image with the scale bar via cv2
        cv2.imwrite(image_path, new)
#

image_compsite_path
/home/lippincm/Documents/ML/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrGasdermin_3_1_im1_im3_im4_composite_image_cell_0.png
1805.54 2140
/home/lippincm/Documents/ML/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrGasdermin_3_1_im1_im3_im4_composite_image_cell_2.png
1805.54 2140
/home/lippincm/Documents/ML/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrGasdermin_3_1_im1_im3_im4_composite_image_cell_3.png
1805.54 2140
/home/lippincm/Documents/ML/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrGasdermin_3_1_im1_im3_im4_composite_image_cell_4.png
1805.54 2140
/home/lippincm/Documents/ML/Interste